# a_my_linear_layer.py

In [1]:
import torch
from torch import nn

In [2]:
class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, input):
        return input @ self.weight.t() + self.bias

- nn.Module을 상속 받아 만든 레이어
- in_features * out_features에 대한 weigth를 갯수를 가지고 out_features에 대한 bias를 가짐
- @ : matmul
- self.weight.t() : 텐서에 대한 transpose

In [5]:
if __name__ == "__main__":
    my_linear = MyLinear(in_features=4, out_features=3)
    sample_input = torch.randn(4)
    output = my_linear(sample_input)

    print("input.shape:", sample_input.shape)
    print("my_linear.weight.shape:", my_linear.weight.shape)
    print("my_linear.bias.shape:", my_linear.bias.shape)
    print("output.shape:", output.shape)
    print(output)

    print("#" * 50, 1)

    my_linear_2 = nn.Linear(in_features=4, out_features=3)
    output_2 = my_linear(sample_input)
    print(output_2)

    assert output.equal(output_2)

    print("#" * 50, 2)

    batch_input = torch.randn(10, 4)
    batch_output = my_linear(batch_input)
    print(batch_output)

    print("#" * 50, 3)

    batch_input_2 = torch.randn(10, 4, 80, 80)
    batch_output_2 = my_linear(batch_input_2)
    print(batch_output.shape)

input.shape: torch.Size([4])
my_linear.weight.shape: torch.Size([3, 4])
my_linear.bias.shape: torch.Size([3])
output.shape: torch.Size([3])
tensor([-0.4203, -3.4312, -0.7370], grad_fn=<AddBackward0>)
################################################## 1
tensor([-0.4203, -3.4312, -0.7370], grad_fn=<AddBackward0>)
################################################## 2
tensor([[ 4.3584, -6.3833, -2.0364],
        [ 0.9813, -1.8385, -0.3435],
        [ 2.3345, -9.0422, -2.4427],
        [ 0.2911,  0.4767,  0.8136],
        [ 3.4141, -1.5411,  0.4961],
        [-1.6599, -1.1402,  0.1145],
        [-1.2370,  1.6302,  2.7581],
        [-4.6392, -1.4896,  1.0107],
        [ 1.5482,  0.8550,  0.7785],
        [ 1.4138, -0.6657,  1.0373]], grad_fn=<AddBackward0>)
################################################## 3


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3200x80 and 4x3)

- my_linear = MyLinear(in_features=4, out_features=3) : 입력값 4개에 3개의 출력
- sample_input = torch.randn(4) : 입력 데이터를 정규분포를 따르는 랜덤한 4개의 값
- output = my_linear(sample_input) : 3개로 나옴
- batch_input_2 = torch.randn(10, 4, 80, 80) -> 이미지 데이터는 넣을 수 없음

# b_my_model.py

In [6]:
import torch
from torch import nn

In [7]:
class MyFirstModel(nn.Module):
    def __init__(self, n_input, n_hidden_unit, n_output):
        super().__init__()

        # Inputs to 1st hidden linear layer
        self.hidden = nn.Linear(n_input, n_hidden_unit)
        self.sigmoid = nn.Sigmoid()

        # Output layer
        self.output = nn.Linear(n_hidden_unit, n_output)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        return x